In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap

## START HERE IF YOU HAVE THE ABOVE FILE ALREADY

In [2]:
df = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/feed_into_pipeline_data.csv')

## Adding Summaries and People **Rankings**

In [3]:
summaries = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/GlobalOriginalsWithSummary.csv')[['combined_title', 'summary']]
people_rankings = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/showsWithPrincipalRankings.csv')[['combined_title', 'director_rank', 'writer_rank', 'actor_rank']]

In [4]:
df = df.merge(summaries, on='combined_title')
df = df.merge(people_rankings, on='combined_title')

## Split the data into movies and tv, then train/test split

In [5]:
#Split movie and tv_show
movies, tv_shows = df[df['is_tv_show']==False], df[df['is_tv_show']==True]

In [6]:
# train test split with stratified sampling
from sklearn.model_selection import train_test_split

tvshows_X_train, tvshows_X_test, tvshows_y_train, tvshows_y_test = train_test_split(tv_shows.drop('is_top10', axis=1), tv_shows['is_top10'], test_size=0.2, random_state=42, stratify=tv_shows['is_top10'])

## Hugging Face PCA

In [7]:
# import summary_transformer_embeddings.csv
summary_embeddings = pd.read_csv('/Users/yenhann/Documents/MIT/15.572 Analytics Lab/Datasets/summary_transformer_embeddings.csv')
summary_embeddings_df = summary_embeddings.copy()

In [8]:
summary_embeddings_df

,summary_embedding_0,summary_embedding_1,summary_embedding_2,summary_embedding_3,summary_embedding_4,summary_embedding_5,summary_embedding_6,summary_embedding_7,summary_embedding_8,summary_embedding_9,...,summary_embedding_374,summary_embedding_375,summary_embedding_376,summary_embedding_377,summary_embedding_378,summary_embedding_379,summary_embedding_380,summary_embedding_381,summary_embedding_382,summary_embedding_383
0,-0.026633,0.018920,-0.055134,0.045714,-0.063023,-0.032902,-0.024073,-0.049662,0.021096,-0.002936,...,0.073990,-0.024292,-0.000128,0.056980,-0.071850,0.032407,0.101245,0.041416,0.107043,-0.010314
1,0.008881,-0.022368,0.007538,0.018812,-0.003038,0.020658,-0.104257,-0.010471,-0.032509,-0.071558,...,-0.009092,0.048180,0.053298,-0.018979,-0.034616,-0.023176,0.066810,-0.137848,0.092657,0.052799
2,-0.023224,-0.003328,-0.020494,0.026740,-0.009808,-0.004658,-0.028434,-0.025144,0.017680,-0.029676,...,0.073531,-0.027467,0.035513,-0.008700,0.048015,0.016851,0.014063,-0.039804,-0.037779,-0.008573
3,-0.019948,0.036421,-0.032200,0.017349,-0.001934,-0.001572,0.068681,-0.014159,-0.022527,0.040237,...,0.090168,0.066829,0.020300,0.105946,-0.012541,0.031277,0.135142,0.099034,-0.108421,-0.074679
4,0.007733,0.069801,-0.040708,0.076647,-0.006824,0.048367,0.059733,-0.014537,-0.030768,-0.005307,...,-0.013327,-0.027364,-0.082106,0.015057,-0.101740,-0.069111,0.029374,-0.031163,0.035734,0.029763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,0.065544,-0.016539,0.012974,-0.012867,0.042465,-0.021782,0.080996,-0.081871,0.007024,-0.049119,...,0.045124,-0.071766,-0.008403,0.036693,0.010890,0.043983,0.011136,-0.017340,0.033225,-0.048873
2222,0.016400,-0.043311,0.054263,0.004811,-0.032548,0.047001,0.047832,-0.020683,-0.009917,-0.056862,...,-0.027186,0.055188,-0.066201,0.048929,-0.124170,-0.050052,0.110961,-0.032043,-0.007126,0.028938
2223,0.065066,-0.001605,0.029295,-0.036900,-0.028460,0.093836,0.075292,-0.038419,-0.066347,-0.012721,...,0.028930,-0.037840,-0.103201,0.142967,-0.044767,0.014078,0.010336,0.031978,0.020591,-0.086957
2224,-0.018001,-0.053426,-0.103662,0.075346,-0.018186,0.045000,0.067063,-0.010451,0.007042,0.041649,...,-0.015691,-0.069962,0.021161,0.045569,-0.052063,0.110053,-0.054747,-0.060268,0.044384,-0.028002


In [48]:
# apply PCA to summary_embeddings_df
from sklearn.decomposition import PCA

nComponents = 200
pca = PCA(n_components=nComponents)
summary_embeddings_pca_df = pca.fit_transform(summary_embeddings_df)

In [49]:
pd.DataFrame(summary_embeddings_pca_df)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.031450,0.247994,-0.030031,-0.204703,0.070657,0.078839,-0.149140,-0.244863,-0.009455,-0.054441,...,0.035677,-0.008985,0.023772,-0.020248,-0.060444,0.021185,0.001361,0.013080,-0.032687,0.073159
1,-0.227742,-0.098685,-0.137212,0.138005,0.122655,-0.020513,0.098578,-0.082827,0.061267,-0.067377,...,-0.030170,0.036172,0.010585,-0.019392,-0.027909,-0.017109,-0.019210,-0.058716,-0.025170,-0.026293
2,-0.117267,0.045462,0.121812,-0.121147,-0.024666,0.059354,0.095709,0.119831,-0.033387,-0.101317,...,-0.041722,0.018839,-0.040326,-0.000397,0.030880,0.024710,-0.030002,-0.016323,-0.021201,0.023950
3,-0.080910,-0.044301,0.098781,0.299777,0.112113,-0.042162,0.060013,-0.137956,0.055825,0.181168,...,-0.017641,-0.034380,0.004945,-0.009422,-0.024506,-0.021349,-0.019080,-0.002773,0.031755,-0.045331
4,-0.042086,0.031152,0.069962,-0.224520,0.303595,-0.039167,-0.174553,-0.107023,-0.025669,-0.095999,...,0.038204,-0.003776,-0.019048,0.029524,0.003365,-0.028068,-0.007635,0.004909,-0.014006,0.025897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,-0.168729,-0.019834,-0.310371,-0.123630,-0.041336,-0.102093,-0.089941,-0.231081,-0.159561,0.016989,...,0.014897,0.004819,0.014700,-0.029804,-0.037253,-0.058195,0.026109,-0.009299,0.041200,-0.027543
2222,-0.212929,-0.019776,-0.023201,0.198410,-0.000942,-0.136511,-0.024948,-0.036193,0.180170,0.058768,...,0.026754,-0.008758,-0.024729,-0.027767,0.043924,0.047197,0.017757,-0.048212,-0.061996,-0.019113
2223,-0.199548,0.103003,-0.122135,-0.103774,-0.051027,-0.113499,-0.104250,-0.320650,-0.134817,-0.003452,...,-0.039060,0.089749,-0.035258,0.033969,-0.005872,-0.042397,0.050430,0.032413,0.014530,-0.015379
2224,0.254890,0.105878,-0.266983,0.275503,-0.063390,-0.139316,-0.020193,-0.011852,-0.036879,-0.051364,...,-0.047882,0.012773,-0.028095,-0.012515,-0.018144,0.038218,0.018847,-0.005048,0.008607,0.032364


In [52]:
print("The total explained variance by", nComponents, "components = ", 100*sum(pca.explained_variance_),"%")

The total explained variance by 200 components =  80.72510992289608 %


In [11]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

In [13]:
rankings_pipeline = Pipeline([
    #('imputer', SimpleImputer(strategy='mean')),
    ('logger', FunctionTransformer(np.log1p))
])

xgb_transformer = ColumnTransformer(
    [
        ("onehot_titleType",OneHotEncoder(handle_unknown='ignore'), ['titleType']),
        ("onehot_release_year",OneHotEncoder(handle_unknown='ignore'), ['release_year']),
        #("bagofwords", CountVectorizer(stop_words='english', min_df=0.02), 'summary'), # One hot encoding
        #("tfidf", TfidfVectorizer(stop_words='english', min_df=0.02), 'summary'),      # TF-IDF
        ("genres_count", CountVectorizer(min_df=0.05), 'genres'),
        #("sentence_transformer", embedder, 'summary'),                                 # hugging face - PCA
        #("imputer", SimpleImputer(strategy='mean'), ['averageRating', 'numVotes', 'runtimeMinutes']),
        ("rankings_pipeline", rankings_pipeline, ['director_rank', 'actor_rank', 'writer_rank']),
    ],
    remainder='passthrough')

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Recreate pipeline using XGBoost

xgb_pipe = Pipeline(
    steps=[
        ('xgb_transformer', xgb_transformer),
        #('normalizer', sklearn.preprocessing.Normalizer()),
        ('xgb', XGBClassifier())
    ]
)

xgb_gs = GridSearchCV(
    xgb_pipe,
    param_grid={
        'xgb__max_depth': [3, 5, 7],
        'xgb__learning_rate': [0.01, 0.1, 0.5],
        'xgb__n_estimators': [100, 200, 300]
    },
    scoring={"AUC": "roc_auc", "Accuracy": "accuracy"},
    refit="AUC",
    cv=5,
    verbose=0,
    n_jobs=-1,
)